<p style="font-family: 'Courier New', Courier, monospace; font-size: 40px; font-weight: bold; color: blue;  text-align: center;">
  LoRaWAN Path Loss Measurements in an Indoor Setting: DATA QUERYING from InfluxdB
</p>

In [2]:
# Libraries/Packages Used:

import pandas as pd  # For data manipulation and handling timezones
from influxdb import InfluxDBClient  # To interact with InfluxDB, a time-series database

In [3]:
# InfluxDB connection parameters
host = 'xx.xx.xx.xx'
port = xxxx
database = 'xxxx'

In [4]:
def fetch_data(start_time, end_time):
    """
    Fetch sensor data from InfluxDB between specified start and end times.
    
    Converts input times from Europe/Berlin to UTC for querying, then back to Europe/Berlin for use.
    Returns data as a pandas DataFrame for analysis.
    """
    client = InfluxDBClient(host=host, port=port)
    client.switch_database(database)

    # Convert input times (Europe/Berlin) to UTC for the query
    start_time_utc = pd.to_datetime(start_time).tz_localize('Europe/Berlin').tz_convert('UTC').strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time_utc = pd.to_datetime(end_time).tz_localize('Europe/Berlin').tz_convert('UTC').strftime('%Y-%m-%dT%H:%M:%SZ')

    # Query with the provided start_time and end_time in UTC
    query = f"SELECT * FROM sensor_data WHERE time >= '{start_time_utc}' AND time <= '{end_time_utc}'"
    
    result = client.query(query)
    df = pd.DataFrame(list(result.get_points()))
    
    if not df.empty:
        # Convert time back to Europe/Berlin timezone
        df['time'] = pd.to_datetime(df['time']).dt.tz_convert('Europe/Berlin')
    
    return df

# Selecting Time Limits for the Data
# Time Limit:
start_time = '2024-08-12 11:00:00'  # Berlin time
end_time = '2024-09-04 10:00:00'  # Berlin time

# Fetching data from InfluxDB
new_df = fetch_data(start_time, end_time)

# Save the combined DataFrame to a local CSV file
new_df.to_csv('combined_measurements_data.csv', index=False)